In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, regularizers

In [2]:
df = pd.read_csv(r"C:\Users\fyl\OneDrive - UWE Bristol\Data Analysis Projects\Python\NLNG Projects.csv")
df

,Project ID,Project Name,Project Type,Task Name,Contractor Type,Labor Count,Task Duration (hours),Task Duration (days),Total Cost (£,Start Date,Location,Complexity Level,season
0,2001,NLNG Gas Pipeline Construction,Infrastructure,Electrical Cabling,General,17,135,16.9,137700,2025-02-24,Abuja,High,Dry season
1,2001,NLNG Gas Pipeline Construction,Infrastructure,Structural Steel Assembly,General,13,378,47.2,560196,2025-08-10,Delta,High,Raining season
2,2001,NLNG Gas Pipeline Construction,Infrastructure,Electrical Cabling,General,9,142,17.8,62622,2025-06-21,Port Harcourt,High,Raining season
3,2001,NLNG Gas Pipeline Construction,Infrastructure,Site Clearance,Specialist,5,588,73.5,126420,2025-07-10,Port Harcourt,Medium,Raining season
4,2001,NLNG Gas Pipeline Construction,Infrastructure,Pile Driving,Specialist,16,430,53.8,660480,2025-03-03,Port Harcourt,Medium,Dry season
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2100,NLNG Plant Expansion,Industrial,Painting & Coating,General,16,211,26.4,239696,2025-05-24,Uyo,High,Raining season
1996,2100,NLNG Plant Expansion,Industrial,Electrical Cabling,Specialist,14,405,50.6,447930,2025-11-09,Port Harcourt,High,Dry season
1997,2100,NLNG Plant Expansion,Industrial,Scaffolding,Specialist,17,727,90.9,1446003,2025-08-01,Delta,High,Raining season
1998,2100,NLNG Plant Expansion,Industrial,Process Equipment Erection,Specialist,22,201,25.1,499686,2025-08-02,Delta,High,Raining season


In [3]:
x = df.drop(columns=['Task Duration (hours)', 
                            'Project ID', 'Task Duration (days)' ,
                            'Start Date'])
y = df['Task Duration (hours)']

In [4]:
categorical_cols = ["Project Name", 
                    "Project Type", "Task Name", 
                    "Contractor Type", "Location", 
                    "Complexity Level", "season"]
numeric_cols = [col for col in x.columns if col not in categorical_cols]

In [5]:
# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False)
x_cat = encoder.fit_transform(x[categorical_cols])

In [6]:
# Scale numeric features
scaler = StandardScaler()
x_num = scaler.fit_transform(x[numeric_cols])

In [7]:
# Combine processed features
x_processed = np.hstack([x_num, x_cat])

In [8]:
# Train/test split
x_train, x_test, y_train, y_test = train_test_split(x_processed, y, test_size=0.2, random_state=42)


In [9]:
# Define the model
model = keras.Sequential([
    layers.Input(shape=(x_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.2),
    layers.Dense(1)  # Predict continuous duration in days
])

In [10]:
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mae', metrics=['mae'])
cb = [
    keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=20, factor=0.5)
]

In [11]:
# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1000, batch_size=32)


Epoch 1/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 432.0731 - mae: 432.0519 - val_loss: 433.5936 - val_mae: 433.5688
Epoch 2/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 357.7121 - mae: 357.6801 - val_loss: 323.9026 - val_mae: 323.8607
Epoch 3/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 216.6586 - mae: 216.6074 - val_loss: 207.5212 - val_mae: 207.4642
Epoch 4/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 175.6894 - mae: 175.6319 - val_loss: 183.3539 - val_mae: 183.2953
Epoch 5/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 159.5907 - mae: 159.5314 - val_loss: 164.7932 - val_mae: 164.7333
Epoch 6/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 147.0759 - mae: 147.0149 - val_loss: 148.5209 - val_mae: 148.4592
Epoch 7/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 138.0231 - mae: 137.9616 - val_loss: 139.6921 - val_mae: 139.6291
Epoch 8/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 134.2026 - mae: 134.1402 - val_loss: 133.8573 - val_mae: 1

Epoch 52/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 101.8766 - mae: 101.7951 - val_loss: 106.5212 - val_mae: 106.4398
Epoch 53/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 98.1392 - mae: 98.0573 - val_loss: 107.7383 - val_mae: 107.6558
Epoch 54/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 99.6994 - mae: 99.6166 - val_loss: 106.3077 - val_mae: 106.2246
Epoch 55/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 100.1656 - mae: 100.0822 - val_loss: 108.6873 - val_mae: 108.6040
Epoch 56/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 100.3259 - mae: 100.2421 - val_loss: 110.1165 - val_mae: 110.0324
Epoch 57/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 96.9197 - mae: 96.8361 - val_loss: 104.6499 - val_mae: 104.5665
Epoch 58/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 99.5432 - mae: 99.4588 - val_loss: 109.5035 - val_mae: 109.4184
Epoch 59/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 97.6231 - mae: 97.5386 - val_loss: 109.7679 - val_mae: 109

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 89.8066 - mae: 89.7068 - val_loss: 103.7195 - val_mae: 103.6195
Epoch 104/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 92.2193 - mae: 92.1190 - val_loss: 101.3493 - val_mae: 101.2493
Epoch 105/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 91.4029 - mae: 91.3023 - val_loss: 100.1212 - val_mae: 100.0209
Epoch 106/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 88.8037 - mae: 88.7026 - val_loss: 104.8604 - val_mae: 104.7589
Epoch 107/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 86.7176 - mae: 86.6161 - val_loss: 100.4048 - val_mae: 100.3032
Epoch 108/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 93.1275 - mae: 93.0258 - val_loss: 106.1948 - val_mae: 106.0928
Epoch 109/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 91.7523 - mae: 91.6506 - val_loss: 103.4417 - val_mae: 103.3397
Epoch 110/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 90.1841 - mae: 90.0819 - val_loss: 106.8847 - val_mae: 106.7823
Epoch 1

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 86.5831 - mae: 86.4673 - val_loss: 99.6614 - val_mae: 99.5464
Epoch 155/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 89.0998 - mae: 88.9841 - val_loss: 102.1993 - val_mae: 102.0834
Epoch 156/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 89.6219 - mae: 89.5061 - val_loss: 106.8544 - val_mae: 106.7383
Epoch 157/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 85.6604 - mae: 85.5442 - val_loss: 102.6747 - val_mae: 102.5585
Epoch 158/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 83.8544 - mae: 83.7377 - val_loss: 101.9950 - val_mae: 101.8783
Epoch 159/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 85.9000 - mae: 85.7833 - val_loss: 100.3061 - val_mae: 100.1893
Epoch 160/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 88.6018 - mae: 88.4845 - val_loss: 102.0900 - val_mae: 101.9732
Epoch 161/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 87.2532 - mae: 87.1361 - val_loss: 100.3943 - val_mae: 100.2776
Epoch 162

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 82.4936 - mae: 82.3635 - val_loss: 101.2333 - val_mae: 101.1034
Epoch 206/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 82.7101 - mae: 82.5796 - val_loss: 104.4227 - val_mae: 104.2923
Epoch 207/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 86.6215 - mae: 86.4910 - val_loss: 110.6352 - val_mae: 110.5042
Epoch 208/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 85.9408 - mae: 85.8101 - val_loss: 106.5314 - val_mae: 106.4005
Epoch 209/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 82.6413 - mae: 82.5103 - val_loss: 106.3199 - val_mae: 106.1884
Epoch 210/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 81.0045 - mae: 80.8729 - val_loss: 106.2701 - val_mae: 106.1382
Epoch 211/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 81.7935 - mae: 81.6613 - val_loss: 103.4866 - val_mae: 103.3546
Epoch 212/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 85.4441 - mae: 85.3123 - val_loss: 110.5995 - val_mae: 110.4668
Epoch 2

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.3639 - mae: 70.1170 - val_loss: 113.4269 - val_mae: 113.1803
Epoch 767/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 69.6641 - mae: 69.4171 - val_loss: 117.7355 - val_mae: 117.4882
Epoch 768/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 73.3007 - mae: 73.0532 - val_loss: 114.2572 - val_mae: 114.0100
Epoch 769/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 72.0386 - mae: 71.7911 - val_loss: 118.6909 - val_mae: 118.4435
Epoch 770/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 73.2884 - mae: 73.0407 - val_loss: 114.5555 - val_mae: 114.3081
Epoch 771/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.1873 - mae: 70.9395 - val_loss: 119.4325 - val_mae: 119.1843
Epoch 772/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 72.0469 - mae: 71.7986 - val_loss: 112.9110 - val_mae: 112.6633
Epoch 773/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.2851 - mae: 71.0369 - val_loss: 117.1411 - val_mae: 116.8928
Epoch 7

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 67.9445 - mae: 67.6894 - val_loss: 125.0289 - val_mae: 124.7731
Epoch 818/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.9003 - mae: 70.6450 - val_loss: 116.4838 - val_mae: 116.2287
Epoch 819/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.9873 - mae: 71.7318 - val_loss: 117.2364 - val_mae: 116.9811
Epoch 820/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 74.3983 - mae: 74.1427 - val_loss: 122.8250 - val_mae: 122.5689
Epoch 821/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.1364 - mae: 70.8807 - val_loss: 112.7208 - val_mae: 112.4655
Epoch 822/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.6271 - mae: 70.3713 - val_loss: 119.8336 - val_mae: 119.5778
Epoch 823/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 73.1835 - mae: 72.9275 - val_loss: 118.7550 - val_mae: 118.4990
Epoch 824/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.5326 - mae: 71.2767 - val_loss: 123.6316 - val_mae: 123.3752
Epoch 8

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.3127 - mae: 70.0502 - val_loss: 113.9935 - val_mae: 113.7314
Epoch 869/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 72.8996 - mae: 72.6370 - val_loss: 113.9290 - val_mae: 113.6670
Epoch 870/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.9004 - mae: 71.6377 - val_loss: 111.1577 - val_mae: 110.8956
Epoch 871/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.1791 - mae: 69.9162 - val_loss: 119.3161 - val_mae: 119.0532
Epoch 872/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 73.8066 - mae: 73.5438 - val_loss: 121.9529 - val_mae: 121.6899
Epoch 873/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 69.1140 - mae: 68.8505 - val_loss: 118.5652 - val_mae: 118.3021
Epoch 874/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.5491 - mae: 70.2857 - val_loss: 125.3896 - val_mae: 125.1256
Epoch 875/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.7896 - mae: 70.5260 - val_loss: 118.5415 - val_mae: 118.2778
Epoch 8

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 68.8779 - mae: 68.6081 - val_loss: 130.6304 - val_mae: 130.3598
Epoch 920/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.5521 - mae: 71.2820 - val_loss: 119.5067 - val_mae: 119.2368
Epoch 921/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 72.1853 - mae: 71.9150 - val_loss: 116.8418 - val_mae: 116.5720
Epoch 922/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 69.9326 - mae: 69.6624 - val_loss: 122.4381 - val_mae: 122.1676
Epoch 923/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.0124 - mae: 70.7418 - val_loss: 127.4043 - val_mae: 127.1332
Epoch 924/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 73.9864 - mae: 73.7156 - val_loss: 125.5326 - val_mae: 125.2616
Epoch 925/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.5573 - mae: 71.2865 - val_loss: 128.6296 - val_mae: 128.3582
Epoch 926/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.8035 - mae: 70.5326 - val_loss: 118.3148 - val_mae: 118.0443
Epoch 9

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 76.0679 - mae: 75.7907 - val_loss: 120.7531 - val_mae: 120.4759
Epoch 971/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 74.3073 - mae: 74.0301 - val_loss: 121.4490 - val_mae: 121.1718
Epoch 972/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 69.2332 - mae: 68.9560 - val_loss: 123.8551 - val_mae: 123.5776
Epoch 973/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 71.8457 - mae: 71.5680 - val_loss: 120.6597 - val_mae: 120.3821
Epoch 974/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 76.5061 - mae: 76.2285 - val_loss: 118.0574 - val_mae: 117.7802
Epoch 975/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 69.0204 - mae: 68.7430 - val_loss: 116.5096 - val_mae: 116.2324
Epoch 976/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.8102 - mae: 70.5327 - val_loss: 131.2246 - val_mae: 130.9461
Epoch 977/1000
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 70.3946 - mae: 70.1165 - val_loss: 115.5071 - val_mae: 115.2295
Epoch 9

In [12]:
# Evaluate the model
loss, mae = model.evaluate(x_test, y_test, verbose=0)
mae = mae/24
print(f"Mean Absolute Error on test data: {mae:.2f} days")

Mean Absolute Error on test data: 5.25 days


In [13]:
user_data = {
    "Project Name": ["NLNG Plant Expansion"],
    "Project Type": ["Industrial"],
    "Task Name": ["Instrumentation"],
    "Contractor Type": ["General"],
    "Labor Count": [20],
    "Total Cost (£": [521840],  # Can include or drop depending on leakage concerns
    "Location": ["Calabar"],
    "Complexity Level": ["Medium"],
    "season": ["Raining season"],
}

# 1) Make sure your encoder can handle new/unseen categories
# (re-fit once with handle_unknown='ignore' before training)
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
x_cat = encoder.fit_transform(x[categorical_cols])  # re-fit once, then keep this encoder

# 2) Ensure your scaler is the same one already fit on training data (you already have 'scaler')

# 3) Create a DataFrame for the new row
user_df = pd.DataFrame(user_data)

# 4) Make sure the user_df has ALL columns expected by your preprocessing
#    (i.e., all categorical_cols and numeric_cols must be present).
#    If any numeric column is missing, create it with a neutral value (e.g., 0) or a sensible default.
for col in numeric_cols:
    if col not in user_df.columns:
        user_df[col] = 0  # or use a better default (e.g., training median)

for col in categorical_cols:
    if col not in user_df.columns:
        user_df[col] = ""  # empty string -> becomes all-zeros in one-hot

# 5) Apply the SAME transforms
user_x_num = scaler.transform(user_df[numeric_cols])
user_x_cat = encoder.transform(user_df[categorical_cols])

# 6) Combine in the SAME order as training
user_x = np.hstack([user_x_num, user_x_cat])

# 7) Predict
pred_hours = model.predict(user_x)[0, 0]
pred_days = int(pred_hours/24)
print(f"Predicted Task Duration: {pred_hours:.1f} hours")
print(f"Predicted Task Duration: {pred_days:.1f} days")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicted Task Duration: 500.6 hours
Predicted Task Duration: 20.0 days
